# Hardware Deployment: Run on Real Quantum Computers

Execute quantum circuits on IBM Quantum (FREE tier available).

**Time:** 20-30 minutes  
**Level:** Advanced

## Prerequisites

1. **Get FREE IBM Quantum account:** https://quantum.ibm.com
2. **Copy your API token** from account settings
3. **Install backend:**
```bash
pip install quantum-debugger[ibm]
```

In [ ]:
from quantum_debugger.backends import IBMQuantumBackend
from quantum_debugger.optimization import compile_circuit
import numpy as np

## Step 1: Connect to IBM Quantum

In [ ]:
# Initialize backend
backend = IBMQuantumBackend()

# Connect with your token
YOUR_TOKEN = "YOUR_IBM_TOKEN_HERE"  # Replace with your token

try:
    backend.connect({'token': YOUR_TOKEN})
    print("Connected to IBM Quantum!")
except Exception as e:
    print(f"Connection failed: {e}")
    print("Get your free token at: https://quantum.ibm.com")

## Step 2: Check Available Devices

In [ ]:
# List available quantum computers
devices = backend.get_available_devices()

print("Available Quantum Computers:")
for device in devices:
    info = backend.get_device_info(device)
    print(f"\n{device}:")
    print(f"  Qubits: {info.get('n_qubits', 'N/A')}")
    print(f"  Status: {info.get('status', 'N/A')}")

## Step 3: Create and Optimize Circuit

Always optimize circuits before running on real hardware!

In [ ]:
# Create Bell state circuit
gates = [
    ('h', 0),           # Hadamard on qubit 0
    ('cnot', (0, 1))    # CNOT: control=0, target=1
]

print(f"Original circuit: {len(gates)} gates")

# Optimize for hardware
optimized_gates = compile_circuit(gates, optimization_level=3)
print(f"Optimized circuit: {len(optimized_gates)} gates")

## Step 4: Execute on Simulator First

Test on simulator before using real hardware (saves queue time).

In [ ]:
# Execute on simulator
print("Running on simulator...")
sim_counts = backend.execute(
    optimized_gates,
    n_shots=1024,
    device='ibmq_qasm_simulator'  # Use simulator
)

print("\nSimulator Results:")
for state, count in sorted(sim_counts.items()):
    print(f"  |{state}>: {count} ({count/1024*100:.1f}%)")

## Step 5: Execute on Real Quantum Computer

**Note:** This uses your free tier credits!

In [ ]:
# Execute on real QPU
print("Submitting to real quantum computer...")
print("This may take a few minutes (queue time)\n")

try:
    qpu_counts = backend.execute(
        optimized_gates,
        n_shots=1024,
        device='ibm_brisbane'  # Or any available device
    )
    
    print("Real QPU Results:")
    for state, count in sorted(qpu_counts.items()):
        print(f"  |{state}>: {count} ({count/1024*100:.1f}%)")
        
except Exception as e:
    print(f"Execution failed: {e}")
    print("This is normal if no devices are available or queue is full.")

## Step 6: Compare Simulator vs Real Hardware

In [ ]:
import matplotlib.pyplot as plt

# Prepare data for plotting
states = ['00', '01', '10', '11']
sim_probs = [sim_counts.get(s, 0)/1024 for s in states]

# If QPU results available
if 'qpu_counts' in locals():
    qpu_probs = [qpu_counts.get(s, 0)/1024 for s in states]
    
    # Plot comparison
    x = np.arange(len(states))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(x - width/2, sim_probs, width, label='Simulator', alpha=0.8)
    ax.bar(x + width/2, qpu_probs, width, label='Real QPU', alpha=0.8)
    
    ax.set_xlabel('Quantum State')
    ax.set_ylabel('Probability')
    ax.set_title('Simulator vs Real Quantum Computer')
    ax.set_xticks(x)
    ax.set_xticklabels([f'|{s}>' for s in states])
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
else:
    print("QPU results not available - showing simulator only")
    
    plt.figure(figsize=(8, 5))
    plt.bar(states, sim_probs, alpha=0.8)
    plt.xlabel('Quantum State')
    plt.ylabel('Probability')
    plt.title('Simulator Results')
    plt.grid(True, alpha=0.3, axis='y')
    plt.show()

## Step 7: Apply Error Mitigation

Real quantum computers have noise - use error mitigation!

In [ ]:
from quantum_debugger.qml.mitigation import apply_readout_mitigation

# This is a simplified example
# In practice, you'd use PEC or CDR from Week 4

if 'qpu_counts' in locals():
    print("Raw QPU Results:", qpu_counts)
    
    # Apply basic readout error mitigation
    # (In real use, calibrate with your device)
    mitigated_counts = qpu_counts.copy()  # Simplified
    
    print("\nAfter mitigation:", mitigated_counts)
    print("\nNote: Use PEC/CDR from Week 4 for production mitigation")
else:
    print("No QPU results to mitigate")

## Best Practices for Hardware

### Before Execution:
1. **Optimize circuits** - Use compilation level 3
2. **Test on simulator** - Catch bugs early
3. **Check device status** - Use available devices
4. **Minimize shots** - Start with fewer shots for testing

### During Execution:
1. **Monitor queue** - Real devices have wait times
2. **Use free tier wisely** - 10 minutes/month limit
3. **Job batching** - Submit multiple jobs together

### After Execution:
1. **Apply error mitigation** - Always for production
2. **Compare with simulation** - Validate results
3. **Save results** - Don't re-run unnecessarily

### Cost Management:
- **IBM Quantum:** FREE tier (10 min/month)
- **AWS Braket:** ~$0.30/task + $0.00035/shot
- **Always estimate costs first** for paid services

## Summary

You learned:
1. Connect to IBM Quantum (free)
2. List available devices
3. Optimize circuits for hardware
4. Execute on simulator and real QPU
5. Compare results
6. Apply error mitigation

## Next Steps

- Try different circuits
- Experiment with error mitigation (Week 4)
- Optimize for specific hardware topologies
- Explore AWS Braket (paid tier)

## Resources

- [IBM Quantum](https://quantum.ibm.com)
- [Hardware Backends Guide](../docs/hardware_backends_guide.md)
- [Error Mitigation Guide](../docs/error_mitigation_guide.md)